# 랭체인 도구로 에이전트 만들기
- LangChain에서 도구를 정의하고 사용하는 방법 확인
- LLM이 스스로 도구가 필요하다고 판단하고 도구 실행을 요청하는 과정 확인
- 모델 호출 -> 도구 실행 -> 모델 재호출

## 라이브러리 불러오기

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

c:\Users\gunwo\anaconda3\envs\TEST\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## API KEY 불러오기

In [2]:
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 감사합니다! 잘 지내고 있어요. 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 12, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY16QEpLRcpe9XCaxNLr72BGixJ9I', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3abc224c-5f9f-4607-91b3-82bd22683037-0', usage_metadata={'input_tokens': 12, 'output_tokens': 22, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 도구 정의
- `@tool` : 이 데코레이터를 파이썬 함수 위에 붙이면 LangChain이 이 함수를 LLM이 사용할 수 있는 도구로 반환함
- `dogString`
  - LLM이 docstring을 읽고 이 도구가 무슨 일을 하는지, 언제 이 도구를 사용해야 하는지 등 판단
  - `Args`: LLM이 timezone과 location에 어떤 값을 넣어야 할지 결정하는 데 사용

In [3]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


## 모델에 도구 바인딩
- llm_with_tools 객체는 이제 도구를 사용할지 말지 스스로 결정

In [4]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

## 1단계 : 모델의 도구 호출 결정

In [5]:
from langchain_core.messages import SystemMessage

# 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY16RlaJoeoqQcmsAxu4cAAdQkZsV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--1435169c-f74f-4699-a564-c06ebd5a849f-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_Cs03MejfQATZFef0NmoNOn00', 'type': 'tool_call'}],

### 문제 2)
lm_with_tools 모델이 가장 처음 반환한 AIMessage에 포함된 핵심 정보는?
- llm_with_tools 모델은 질문에 대한 것을 학습을 하지 못해서 질문에 있는 현재 시간을 학습을 하지 못해 대답을 못하는 것입니다. 

## 2단계 : 도구 실행
- 실제 랭체인 사용 시에는 LangChain 프레임워크가 이 1~3단계를 랭체인 에이전트로 자동화함
- `AgentExecutor`에 도구가 바인딩된 LLM과 실제 실행할 도구 목록만 전달해주면 됨

In [6]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-11-04 11:30:28 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY16RlaJoeoqQcmsAxu4cAAdQkZsV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--1435169c-f74f-4699-a564-c06ebd5a849f-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_Cs03MejfQATZFef0NmoNOn00', 'type': 'tool_call'}

## 3단계 : 모델의 최종 답변

In [7]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 현재 2025년 11월 4일 11시 30분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 192, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY16Ssd6f1P2J1sC6O2LQre37vqFF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3e14248b-b836-4744-8588-5f877adc2af7-0', usage_metadata={'input_tokens': 192, 'output_tokens': 23, 'total_tokens': 215, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})